# Entanglement-assisted benchmarking

- With parallel CNOT



In [7]:
import numpy as np
import sys, json, copy, time, pickle, random
from concurrent.futures import ThreadPoolExecutor
import qiskit
from qiskit import IBMQ, QuantumCircuit, execute
from qiskit.providers.aer import StatevectorSimulator, AerSimulator
from qiskit.providers.aer.noise import NoiseModel, pauli_error, amplitude_damping_error, ReadoutError,depolarizing_error
# from qiskit.providers.ibmq.managed import IBMQJobManager, ManagedJobSet
# from qiskit.providers.ibmq.apiconstants import ApiJobShareLevel
from qiskit.quantum_info.operators.symplectic import pauli
# from qubit_map import my_qubit_maps
import matplotlib.pyplot as plt
from scipy.stats import sem, unitary_group
from scipy import sparse
# import EAB_submit, EAB_process
# import EAB_submit_2 as EAB_submit
# import EAB_process
from statistics import stdev
import itertools
from qiskit.compiler import transpile

from qiskit.quantum_info import Pauli

sys.path.append(r"/Users/yzhu/yzhu_work/gates projects/EAB/EAB_code from Senrui")
import EAB_submit, EAB_process
from qubit_map import my_qubit_maps

## Additional functions

In [8]:
## Additional functions

def int_to_pauli(i,n):
    p = np.base_repr(i,base=4)
    p = '0'*(n-len(p)) + p
    p = p.replace('0','I').replace('1', 'X').replace('2', 'Y').replace('3', 'Z')
    return p

def commute(p,q):
    c = 1
    n = len(p)
    for i in range(n):
        if p[i] != 'I' and q[i] != 'I':
            if p[i] != q[i]:
                c *= -1
    return c

def fidelity_to_error(pauli_fidelity,n):
    N = 4**n
    pauli_error = {}
    for i in range(N):
        p = int_to_pauli(i,n)
        pauli_error[p] = 0
        for j in range(N):
            q = int_to_pauli(j,n)
            pauli_error[p] += pauli_fidelity[q] * commute(p,q) / N
    return pauli_error
    

## Data generation

In [13]:

use_density_matrix = False # density matrix based / measurement based simulation

# choose one
use_ibmq = False
use_stabilizer_simulator = True # whether stabilizer simulator is used (valid only for Pauli noise)
use_density_matrix_sample = False # use density matrix simulation, but returns samples
use_state_vector_sample = False # use state vector simulation, which returns samples

# parameters: n, Lrange, C, shots
use_readout_error = True
shots = 1000
n = 2 # num of qubit
n_total = 2*n
# Lrange = list(range(2,39,4)) # len = 10
Lrange = [2**x for x in range(1,6)]
C = 20
batch = 1
gset = "Pauli"
q = my_qubit_maps['local']
repeat = [1 for k in Lrange] # not used
periodic = False # not used

# For specific IBM device structure
# TODO: put these in qubit_map.py
# q = {
#     0:0,
#     2:1,
#     1:3,
#     3:4,
#     4:2
# }
# q = {
#     0:0,
#     2:1,
#     1:2,
#     3:3,
#     4:4
# }

clifford_layer = 'XX'
# clifford_layer = 'Id'

# Simulation parameters
eps = 0.01# two parallel Cnot ~ 2%
# eps_amp = 0.025 # SPAM error at CNOT
eps_readout = 0.005 # readout bitflip ~ 1%
# eps_cross = 0.025 # SPAM error at CNOT

noise_model = NoiseModel()
# # noise_model.add_all_qubit_quantum_error(pauli_error([('I',1-eps),('X',eps)]),['id', 'x', 'y','z'])
# noise_model.add_all_qubit_quantum_error(pauli_error([('II',(1-eps)**2),('XX',eps**2),('IX',eps*(1-eps)),('XI',eps*(1-eps))]),['cx'])
#depolarizing error
error = depolarizing_error(0.1, 2)
noise_model.add_quantum_error(error, ['cx'], [0,2])
noise_model.add_quantum_error(error, ['cx'], [1,3])
print (noise_model)

# # phase/amplitude damping error
# amp_noise_1q = amplitude_damping_error(eps_amp)
# noise_model.add_all_qubit_quantum_error(amp_noise_1q.tensor(amp_noise_1q),['cx'])

# # crosstalk between CNOTs (choose it to be ZZ)
# crosstalk_noise = pauli_error([('II',1-eps_cross),('ZZ',eps_cross)])
# for i in range(n-2):
#     noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [i,i+1], [i+1,i+2])
# noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [n-2,n-1], [n-1,0])
# noise_model.add_nonlocal_quantum_error(crosstalk_noise, ['cx'], [n-1,0], [0,1])

# readout error
if use_readout_error is True:
    p0given1 = eps_readout
    p1given0 = eps_readout
    readout_noise_1q = ReadoutError([[1 - p1given0, p1given0], [p0given1, 1 - p0given1]])
    noise_model.add_all_qubit_readout_error(readout_noise_1q)



if use_ibmq is False:
    filename = 'sim_eab_cnot_2022oct'
else:
    filename = 'exp_eab_cnot_2022oct'
    job_manager = IBMQJobManager()



if use_density_matrix or use_density_matrix_sample:
    backend = AerSimulator(method='density_matrix', noise_model=noise_model)
elif use_state_vector_sample:
    backend = AerSimulator(method='statevector', noise_model=noise_model)
elif use_stabilizer_simulator:
    backend = AerSimulator(method='stabilizer', noise_model=noise_model)
elif use_ibmq:
    provider = IBMQ.load_account()
    backend = provider.get_backend('ibmq_manila')
else:
    assert 1==0

print("EAB, ", "n=%d" % n)
print("simulation method:", backend.configuration().description)



data = {}
token = ''.join(random.choice([str(j) for j in range(10)]) for i in range(10))
filename += '_' + token


# generate EAB circuit:
#Lrange: depth
#C=sample size per depth
# batch: 1??
#qubit_map=
#gset= Pauli
#repeat=[1,1,1,1,1]??
# periodic=False??
eab_data, eab_circ_all = EAB_submit.submit_eab(n,n_total,clifford_layer = clifford_layer,Lrange=Lrange,C=C,batch=batch, qubit_map=q,gset=gset,repeat=repeat,periodic=periodic,use_density_matrix=use_density_matrix)
print("created %d circuits" % len(eab_circ_all[0]))
data_batch = eab_data["batch_0"]
print("length of data_batch %d" %len(data_batch))

print(eab_circ_all[0][0]) # print a typical example
print (eab_circ_all[0][0].qasm())
# print(transpile(eab_circ_all[0][0],basis_gates=backend.configuration().basis_gates))
# print(transpile(eab_circ_all[0][0],optimization_level=3,basis_gates=backend.configuration().basis_gates,coupling_map=backend.configuration().coupling_map,initial_layout=q))

# sys.exit(0)

if use_density_matrix is True:
    job = backend.run(eab_circ_all[0], shots=1, max_parallel_experiments=0) 
elif use_ibmq is False:
    job = backend.run(eab_circ_all[0], shots=shots, max_parallel_experiments=0) 
else:
    job = backend.run(transpile(eab_circ_all[0],basis_gates=backend.configuration().basis_gates), shots=shots) 
result = job.result()
eab_data["result"] = [result.to_dict()]

eab_data["parameters"] = {}
eab_data["parameters"]['n'] = n 
eab_data["parameters"]['n_total'] = n_total
eab_data["parameters"]['shots'] = shots 
eab_data["parameters"]['Lrange'] = Lrange 
eab_data["parameters"]['C'] = C
eab_data["parameters"]['eps_readout'] = eps_readout
eab_data["parameters"]['repeat'] = repeat

eab_data['token'] = token

# if use_ibmq is False:
#     eab_data["result"] = [result]
# else:
#     eab_data["job_set_id"] = job_set_id

# test: data saving
# print(eab_data)
print(token)
# open ('data/' + filename, 'x')
with open('/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/EAB(code from Senrui)/data/' + filename, 'wb') as outfile:
    pickle.dump(eab_data, outfile)

# print(token)

NoiseModel:
  Basis gates: ['cx', 'id', 'rz', 'sx']
  Instructions with noise: ['cx']
  Qubits with noise: [0, 1, 2, 3]
  Specific qubit errors: [('cx', (0, 2)), ('cx', (1, 3))]
EAB,  n=2
simulation method: A C++ Clifford stabilizer simulator with noise
created 100 circuits
length of data_batch 100
     ┌───┐           ░ ┌───┐┌───┐┌───┐┌───┐┌───┐     ┌───┐┌───┐┌───┐ ░ ┌───┐»
q_0: ┤ H ├──■────────░─┤ Z ├┤ Z ├┤ H ├┤ X ├┤ Z ├──■──┤ S ├┤ H ├┤ X ├─░─┤ X ├»
     ├───┤  │        ░ ├───┤└───┘└───┘└───┘└───┘┌─┴─┐├───┤├───┤├───┤ ░ ├───┤»
q_1: ┤ H ├──┼────■───░─┤ I ├────────────────────┤ X ├┤ H ├┤ S ├┤ H ├─░─┤ Z ├»
     └───┘┌─┴─┐  │   ░ └───┘                    └───┘└───┘└───┘└───┘ ░ └───┘»
q_2: ─────┤ X ├──┼───░──────────────────────────────────────────────────────»
          └───┘┌─┴─┐ ░                                                      »
q_3: ──────────┤ X ├─░──────────────────────────────────────────────────────»
               └───┘ ░                                                      

/var/folders/6l/f5pw0tyx785dhvmqpb7282h00000gn/T/ipykernel_78803/3188363930.py:122: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  print (eab_circ_all[0][0].qasm())


4993948144


In [14]:
# for item in (eab_data["result"][0]['results']):
#     print (item)
# print (eab_data["result"][0]['results'][0]['data']['counts'])
# print (eab_data["result"][0]['results'][1]['data']['counts'])
# from qiskit.result import Result
# print ((eab_data["batch_0"][0]))
# print ((eab_data["batch_0"][99]))

# result_batch = Result.from_dict(eab_data["result"][0])
outcomes = result.get_counts(1)
# # print (outcomes.items()["0010"])
# print (outcomes)
print (outcomes)
print (result)

{'1010': 4, '1000': 3, '0000': 3, '1001': 3, '1011': 3, '1111': 37, '1101': 35, '0110': 49, '0010': 42, '0111': 703, '0001': 4, '0011': 49, '0101': 53, '1110': 6, '1100': 6}
Result(backend_name='aer_simulator', backend_version='0.13.2', qobj_id='', job_id='84d610eb-c443-4a62-be92-62ef1c23f9e8', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0xe': 4, '0x0': 3, '0xb': 4, '0xa': 1, '0x2': 41, '0xf': 51, '0xd': 38, '0x3': 51, '0x1': 3, '0x6': 40, '0xc': 2, '0x4': 4, '0x7': 711, '0x8': 3, '0x5': 44}), header=QobjExperimentHeader(creg_sizes=[['c', 4]], global_phase=0.0, memory_slots=4, metadata={}, n_qubits=4, name='circuit-1130', qreg_sizes=[['q', 4]]), status=DONE, seed_simulator=706845717, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 1, 'parallel_shots': 1, 'noise': 'circuit', 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [0, 1, 2, 3], 'd

## Data analysis

In [15]:
# import EAB_process
'''Use these if read from file'''
# token = "1011327761"
# filename = "sim_eab_cnot_2022oct_" + token
# with open('data/' + filename, 'rb') as infile:
#     eab_data = pickle.load(infile)



n = eab_data["parameters"]['n']
n_total = eab_data["parameters"]['n_total'] 
shots = eab_data["parameters"]['shots'] 
Lrange = eab_data["parameters"]['Lrange']
C = eab_data["parameters"]['C'] 
eps_readout = eab_data["parameters"]['eps_readout'] 
repeat = eab_data["parameters"]['repeat']

'''Specify a set of Pauli you want to estimate'''
pauli_request_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = n)] #full

# pauli_request_list = ['XX']

eab_result = EAB_process.process_EAB(batch, Lrange, eab_data, pauli_request_list, repeat=repeat, periodic=periodic,use_density_matrix=use_density_matrix)

# fidelity_list[1..512] = [0.99,0.98,...]

raw_fidelity_list = eab_result["fidelity_list"]


# sys.exit(0)

fidelity_list = {}
stdev_list = {}

for pauli_label in pauli_request_list:
    if(pauli_label == 'I'*n):
        fidelity_list[pauli_label] = 1.0
        stdev_list[pauli_label] = 0.0
    else:
        alpha, alpha_err = EAB_process.fit_EAB(Lrange, raw_fidelity_list[pauli_label])
        fidelity_list[pauli_label] = alpha
        stdev_list[pauli_label] = alpha_err
# print(fidelity_list)

# print(stdev_list)
print("Parameters: n = %d, C = %d, " % (n,C), "L = ", str(Lrange))

if use_density_matrix:
    print("Density matrix based simulation")
else:
    print("Measurement based simulation, shots = %d" % shots)

if use_readout_error is True:
    print("Measurement bitflip rate = %f" % eps_readout)
else:
    print("No readout error")

# Average fidelity
print("Total error = ", 1-np.mean(list(fidelity_list.values())))

print("Label / Pauli infidelity / Standard deviation")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])
# print('Effective noise rate = ' + str(1-np.average(list(fidelity_list.values()))))



Parameters: n = 2, C = 20,  L =  [2, 4, 8, 16, 32]
Measurement based simulation, shots = 1000
Measurement bitflip rate = 0.005000
Total error =  0.0002657313093568092
Label / Pauli infidelity / Standard deviation
II 0.0 0.0
XI 0.000193030245474346 0.00019150951581744578
YI 0.00023362362107393064 0.0001816377157887917
ZI 0.0001751770786089235 0.00015438765946348376
IX 4.658119125733329e-05 0.0001929556380024501
XX 0.0005979852625973558 0.000289620789310067
YX 0.0003346363703171651 0.0002848003853331868
ZX 0.00045399978057203416 0.0002863908313678095
IY -2.0899194293377832e-05 0.00014969948419134678
XY 0.00029803932045935344 0.00029795191119453196
YY 0.0004143347543292819 0.0002661437333469234
ZY 0.0002635817198707313 0.00026387067061394077
IZ 3.770894872523467e-05 0.00017638822278876417
XZ 0.0005098834169818822 0.0002786324900994256
YZ 0.00047631168248785105 0.00029712718826851856
ZZ 0.00023770675124767937 0.00024348739111931727


Note that, the estimation suffers from some degeneracy. E.g., $\lambda_{IZ}$ and $\lambda_{ZZ}$ cannot be individually addressed. Only their geometric mean is estimated and reported. This issue is detailed in [The learnability of Pauli noise](https://arxiv.org/abs/2206.06362).

In [6]:
error_list = fidelity_to_error(fidelity_list,n)
print("Label / Pauli error rates")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], error_list[pauli_label])

Label / Pauli error rates
II 0.9802661249785733
XI 0.004857924649527896
YI 2.4839851629224197e-05
ZI -2.980065682001154e-05
IX 0.009957296894477975
XX 0.004925705828500979
YX 2.4943350088332816e-05
ZX -1.4751988418465478e-05
IY 1.6853337303168736e-05
XY 2.7689817947013085e-06
YY -8.757986503654669e-07
ZY -1.8228942176845564e-06
IZ -3.079801564742635e-05
XZ 2.46329051621641e-06
YZ -2.7910185557838973e-06
ZZ 1.9192098977549588e-06


Similarly, in the estimation $\hat{p}_{XI}=\hat{p}_{XX}=0.0025$ due to degeneracy.

In [7]:
# True value for this specific noise model
def f_true(P):
    ans = 0.0
    for Pi in P:
        if Pi == 'Z' or Pi == 'Y':
            ans += 2*eps
    return ans
def p_true(P):
    if P == "XI" or P == "IX":
        return eps
    else:
        return 0.0

print("Label / True infidelity/ True Pauli error")
for P in pauli_request_list:
    print(P[::-1]," ",f_true(P)," ",p_true(P))

Label / True infidelity/ True Pauli error
II   0.0   0.0
XI   0.0   0.01
YI   0.02   0.0
ZI   0.02   0.0
IX   0.0   0.01
XX   0.0   0.0
YX   0.02   0.0
ZX   0.02   0.0
IY   0.02   0.0
XY   0.02   0.0
YY   0.04   0.0
ZY   0.04   0.0
IZ   0.02   0.0
XZ   0.02   0.0
YZ   0.04   0.0
ZZ   0.04   0.0


In [8]:
plot_dic={}
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], (1-fidelity_list[pauli_label])-f_true(pauli_label))
    plot_dic[pauli_label[::-1]]=(1-fidelity_list[pauli_label])-f_true(pauli_label)

II 0.0
XI 3.3201099060020667e-06
YI -0.0005111871584372145
ZI -0.00033004173029758596
IX -2.4565815118782197e-05
XX -6.963699108464283e-06
YX -0.0005570673255199492
ZX -0.0003608693658354416
IY 0.009727975101719095
XY 0.009694016999577743
YY -0.010444557418704058
ZY -0.010403352430777467
IZ 0.009820235421757268
XZ 0.009793584856456001
YZ -0.010338551447318066
ZZ -0.010319975755474907


In [9]:
print (plot_dic)

{'II': 0.0, 'XI': 3.3201099060020667e-06, 'YI': -0.0005111871584372145, 'ZI': -0.00033004173029758596, 'IX': -2.4565815118782197e-05, 'XX': -6.963699108464283e-06, 'YX': -0.0005570673255199492, 'ZX': -0.0003608693658354416, 'IY': 0.009727975101719095, 'XY': 0.009694016999577743, 'YY': -0.010444557418704058, 'ZY': -0.010403352430777467, 'IZ': 0.009820235421757268, 'XZ': 0.009793584856456001, 'YZ': -0.010338551447318066, 'ZZ': -0.010319975755474907}


In [10]:
# plot_c100={}
# plot_c100=plot_dic

In [11]:
# plot_c30={}
# plot_c30=plot_dic

In [12]:
# plot_c20={}
# plot_c20=plot_dic

In [13]:
# plot_c10={}
# plot_c10=plot_dic

In [14]:
# plot_c5={}
# plot_c5=plot_dic

In [15]:
# plot_c3={}
# plot_c3=plot_dic

In [16]:
plt.figure(figsize=(15,10))
plt.plot(list(plot_c100.keys()),list(plot_c100.values()),"*",markersize=10)
plt.plot(list(plot_c30.keys()),list(plot_c30.values()),"*",markersize=10)
plt.plot(list(plot_c20.keys()),list(plot_c20.values()),"*",markersize=10)
plt.plot(list(plot_c10.keys()),list(plot_c10.values()),"*",markersize=10)
plt.plot(list(plot_c5.keys()),list(plot_c5.values()),"*",markersize=10)
# plt.plot(list(plot_c3.keys()),list(plot_c3.values()),"*",markersize=10)
plt.legend(["C=100","C=30","C=20","C=10","C=5"])
plt.ylabel("f_measure - f_true",fontsize=20)
plt.savefig("eps=0.01 single CNOT EAB.pdf",format="pdf",dpi=500)

NameError: name 'plot_c100' is not defined

<Figure size 1080x720 with 0 Axes>

In [ ]:
print (eab_result)